In [1]:
import requests
import time
import csv
import pandas as pd
import os
from dotenv import load_dotenv

In [12]:
load_dotenv()
api_key = os.getenv('ETHERSCAN_API_KEY')
eth_chain_id = 1
arb_chain_id = 42161
bas_chain_id = 8453
opt_chain_id = 10
pol_chain_id = 137

def get_transfers_in_block_range(start_block, end_block, contract_address, chain_id):
    transfers = []
    url = f"https://api.etherscan.io/v2/api?chainid={chain_id}"
    
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": contract_address,
        "startBlock": start_block,
        "endBlock": end_block,
        "sort": "asc",
        "apikey": api_key
    }
    
    response = requests.get(url, params=params)
    print(response.json())
    data = response.json()
    
    if 'result' in data:
        for transfer in data['result']:
            transfer_info = {
                'timestamp': transfer['timeStamp'],
                'value': transfer['value'],
                'from': transfer['from'],
                'to': transfer['to']
            }
            transfers.append(transfer_info)
    
    return transfers

In [ ]:
get_transfers_in_block_range(0, 100, )

In [3]:
def save_transfers_to_csv(transfers, filename):
    print("Saving")
    keys = transfers[0].keys()
    with open(filename, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(transfers)

In [4]:
def get_all_transfers_in_chunks(start_block, end_block, contract_address, block_chunk_size=10000):
    all_transfers = []
    current_block = start_block
    
    while current_block < end_block:
        next_block = min(current_block + block_chunk_size, end_block)
        print(f"Querying transfers from block {current_block} to {next_block}...")
        
        transfers = get_transfers_in_block_range(current_block, next_block, contract_address)
        all_transfers.extend(transfers)
        
        current_block = next_block + 1
    
    return all_transfers

In [ ]:
# transfers = get_all_transfers_in_chunks(15412117, 20866918, "0x3Afdc9BCA9213A35503b077a6072F3D0d5AB0840", block_chunk_size=1000000)
transfers = get_transfers_in_block_range(15412117, 20866918, "0xA17581A9E3356d9A858b789D68B4d866e593aE94", eth_chain_id)
# save_transfers_to_csv(transfers, "cWETHv3.csv")

In [ ]:
# folder_path = '../data/base'

# all_addresses = set()

# for filename in os.listdir(folder_path):
#     if filename.endswith('.csv') and filename != 'distinct_addresses.csv':
#         file_path = os.path.join(folder_path, filename)
#         df = pd.read_csv(file_path)
        
#         df = df[df['timestamp'] > 1696118399]

#         all_addresses.update(df['from'].unique())
#         all_addresses.update(df['to'].unique())

# distinct_addresses_df = pd.DataFrame(list(all_addresses), columns=['address'])

# distinct_addresses_df.to_csv('../data/base/distinct_addresses_1_year.csv', index=False)